In [ ]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy.thermo.thermoengine import submit
from rmgpy.thermo import ThermoData
from copy import deepcopy
from rmgpy import settings

from IPython.display import display
import os

In [ ]:
path = settings['database.directory']

In [ ]:
# Enter your kinetics library or libraries
reaction_libraries = ['Surface/CPOX_Pt/Deutschmann2006']
# Enter your kinetics family or families
families = "None"

In [ ]:
# optional - choose metal binding energies for surface library
metal = 'Pt111'
# optional - choose to display the reactions that are fixed
display_reactions = False
# optioal - choose which reactions to correct barrier
prompt = True
show_thermo = True

In [ ]:
# Load the database

database = RMGDatabase()
database.load(
    path,
    thermo_libraries=['2-BTP','primaryThermoLibrary','Klippenstein_Glarborg2016', 'BurkeH2O2', 'thermo_DFT_CCSDTF12_BAC', 'DFT_QCI_thermo',
                         'primaryThermoLibrary', 'primaryNS', 'NitrogenCurran', 'NOx2018', 'FFCM1(-)',
                         'SulfurLibrary', 'SulfurGlarborgH2S', 'SABIC_aromatics',
                     'SurfaceThermoPt111'], # can add more
    transport_libraries=[],
    reaction_libraries=reaction_libraries,
    seed_mechanisms=[],
    kinetics_families= families,
    kinetics_depositories=[],
    statmech_libraries=[],
    depository=False,
    solvation=False,
    surface=True,
    testing=False,
)
if metal:
    database.thermo.set_binding_energies(metal)

In [ ]:
def display_prompt():
    
    success = False
    while not success:
        choice = input('Would you like to correct this barrier? (y/n) ')
        if choice in ['y', 'n', '']:
            success = True
        else:
            print('Invalid choice.')

    if choice in ['y','']:
        return True
    elif choice == 'n':
        return False

In [ ]:
def display_thermo(reaction):
    print("-"*50,"THERMO","-"*50)
    for spcs in reaction.reactants + reaction.products:
        print("\t" + spcs.label)
        print("\t\t " + spcs.thermo.comment)
        print(f"\t\t E0: {spcs.thermo.E0.value_si/4184:.2f} kcal/mol")
        print(f"\t\t H298: {spcs.thermo.get_enthalpy(298)/4184:.2f} kcal/mol")
        print(f"\t\t S298: {spcs.thermo.get_entropy(298)/4.184:.2f} cal/mol/K")
    print("-"*108)

In [ ]:
# fix barrier for kinetics families training reactions

for family_name, family in database.kinetics.families.items():
    library = family.get_training_depository()
    for label,entry in library.entries.items():
        if entry.item.reversible:
            reaction = entry.item
            reaction.kinetics =  deepcopy(entry.data)
            for spcs in reaction.reactants + reaction.products:
                if not spcs.thermo:
                    labels = spcs.molecule[0].get_all_labeled_atoms()
                    spcs.molecule[0].clear_labeled_atoms()
                    spcs.generate_resonance_structures()
                    submit(spcs)
                    for label,atom in labels.items():
                        if isinstance(atom,list):
                            for a in atom:
                                a.label=label
                        else:
                            atom.label = label
            reaction.fix_barrier_height()
            if "Ea raised" in reaction.kinetics.comment:
                print("="*49,"REACTION","="*49)
                print(f'{label} {reaction}')
                if display_reactions:
                    display(reaction)
                print(reaction.kinetics)
                response = True
                if show_thermo:
                    display_thermo(reaction)
                if prompt:
                    response = display_prompt()
                if response:
                    print("*** Barrier corrected")
                    n_reactions_fixed += 1
                    entry.data = reaction.kinetics
                else:
                    print("*** Barrier NOT corrected")
    print('*' * 100)
    print(f"FIXED {n_reactions_fixed} barriers in {family_name}")
    print('*'* 100)

In [ ]:
# save training reactions
for family_name, family in database.kinetics.families.items():
    library = family.get_training_depository()
    lib_path = os.path.join(path,'kinetics','families',library.name,'reactions.py')
    library.save(lib_path)

In [ ]:
# fix barrier for library reactions

for library_name, library in database.kinetics.libraries.items():
    n_reactions_fixed = 0
    for label,entry in library.entries.items():
        if entry.item.reversible:
            reaction = entry.item
            reaction.kinetics = deepcopy(entry.data)
            for spcs in reaction.reactants + reaction.products:
                if not spcs.thermo:
                    submit(spcs)
            reaction.fix_barrier_height()
            if "Ea raised" in reaction.kinetics.comment:
                print("="*49,"REACTION","="*49)
                print(f'{label} {reaction}')
                if display_reactions:
                    display(reaction)
                print(reaction.kinetics)
                response = True
                if show_thermo:
                    display_thermo(reaction)
                if prompt:
                    response = display_prompt()
                if response:
                    print("*** Barrier corrected")
                    entry.data = reaction.kinetics
                    n_reactions_fixed += 1
                else:
                    print("*** Barrier NOT corrected")
    print('*' * 100)
    print(f"FIXED {n_reactions_fixed} barriers in {library_name}")
    print('*'* 100)

In [ ]:
# save libraries
for library_name, library in database.kinetics.libraries.items():
    lib_path = os.path.join(path,'kinetics','libraries',library_name,'reactions.py')
    library.save(lib_path)